In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain --upgrade

In [ ]:
pip install boto3 --upgrade

In [2]:
import boto3
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document


In [3]:
kendraIndexId = "063e46f7-1953-4503-a46c-72aa1ddf826f"

kendra_retriever = AmazonKendraRetriever(
    index_id= kendraIndexId
)


In [4]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [5]:
llm2 = SagemakerEndpoint(
endpoint_name="hf-llm-falcon-40b-instruct-bf16-2023-06-23-19-34-33-102",
#endpoint_name="hf-llm-falcon-7b-bf16-2023-06-24-20-08-14-262",
#endpoint_name="hf-llm-falcon-40b-bf16-2023-06-24-20-20-44-608",
model_kwargs={
     "parameters" : {"do_sample": False,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 400
              }},
region_name="us-east-1",
content_handler=content_handler
)


In [6]:
llm_query= "how do I fix a broken screen"

prompt_template = """
  {context}
  >>QUESTION<<: {question}
  >>ANSWER<<:"""
  
PROMPT = PromptTemplate(
      template=prompt_template, input_variables=["context", "question"]
  )
  
chain = load_qa_chain(llm=llm2, prompt=PROMPT)

  

In [7]:
docs = kendra_retriever.get_relevant_documents(llm_query)
print(docs)

[Document(page_content="Document Title: HT212787 \nDocument Excerpt: \nKeep holding the button (or buttons) until you see the recovery mode screen: 5. Locate your iPad on your computer. When you see the option to Restore or Update, choose Update. Your computer will try to reinstall the software without erasing your data. Wait while your computer downloads the software for your iPad. If the download takes more than 15 minutes and your iPad exits the recovery mode screen, let the download finish, then repeat step 3. 6. After the Update or Restore completes, set up your iPad. ## Get more help * Learn what to do if you see an error when you update or restore your iPad. * If you can’t update or restore your iPad with recovery mode, you can't use recovery mode because of broken buttons, or your iPad is stuck in Activation Lock, contact Apple Support.\n", metadata={'source': 'https://s3.us-east-1.amazonaws.com/kendradatasourceforfsdemo/HT212787.txt', 'title': 'HT212787', 'excerpt': "Keep hold

In [8]:
output = chain({"input_documents":docs, "question": llm_query}, return_only_outputs=False)
print(output)


{'input_documents': [Document(page_content="Document Title: HT212787 \nDocument Excerpt: \nKeep holding the button (or buttons) until you see the recovery mode screen: 5. Locate your iPad on your computer. When you see the option to Restore or Update, choose Update. Your computer will try to reinstall the software without erasing your data. Wait while your computer downloads the software for your iPad. If the download takes more than 15 minutes and your iPad exits the recovery mode screen, let the download finish, then repeat step 3. 6. After the Update or Restore completes, set up your iPad. ## Get more help * Learn what to do if you see an error when you update or restore your iPad. * If you can’t update or restore your iPad with recovery mode, you can't use recovery mode because of broken buttons, or your iPad is stuck in Activation Lock, contact Apple Support.\n", metadata={'source': 'https://s3.us-east-1.amazonaws.com/kendradatasourceforfsdemo/HT212787.txt', 'title': 'HT212787', '